<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
from pymystem3 import Mystem
from tqdm import notebook
from tqdm.notebook import tqdm
import re
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
RANDOM_STATE =42

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv', index_col=0)

In [3]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [7]:
tqdm.pandas()
m = Mystem()

In [9]:
# custom_stopwords = set(nltk_stopwords.words('english')) | {
#     'would', 'could', 'should', 'might', 'may', 'also', 'us', 'get', 'go', 'like'
# }

In [10]:
# def clear_text(text):
#     """Улучшенная очистка текста"""
#     text = text.lower()  
#     text = re.sub(r'[^a-zA-Zа-яА-Я\s]', ' ', text)  
#     text = re.sub(r'\s+', ' ', text).strip()  
#     return text

# def lemmatize(text):
#     cleaned = clear_text(text)
#     lemmas = m.lemmatize(cleaned)
#     lemm_text = "".join(lemmas).strip()
#     words = [word for word in lemm_text.split() if word not in custom_stopwords]
#     return " ".join(words)

In [8]:
def clear_text(text):
    """Очистка текста от не-буквенных символов"""
    tmp = re.sub(r'[^a-zA-Zа-яА-Я]', ' ', text)
    tmp_lst = tmp.split()
    return ' '.join(tmp_lst)

def lemmatize(text):
    """Лемматизация очищенного текста"""
    cleaned = clear_text(text)
    lemm_list = m.lemmatize(cleaned)
    lemm_text = "".join(lemm_list).strip()
    return lemm_text


In [9]:
df['clear_lemma'] = df['text'].progress_apply(lemmatize)

  0%|          | 0/159292 [00:00<?, ?it/s]

In [10]:
df.head()

,text,toxic,clear_lemma
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He matches this background colour I m se...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not trying to edit war It s...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestions on impr...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...


In [11]:
X = df['clear_lemma']
y = df['toxic']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=RANDOM_STATE)

In [13]:
X_train = X_train.values.astype('U')
X_test = X_test.values.astype('U')

In [14]:
nltk.download('stopwords')
stopwords = list(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
tfidf = TfidfVectorizer(
    stop_words=stopwords,
    )

In [17]:
X_train_tfidf = tfidf.fit_transform(X_train)

In [18]:
X_train_tfidf.shape

(119469, 142320)

In [20]:
model = LogisticRegression(solver="saga", max_iter=500, random_state=RANDOM_STATE)


In [21]:
with tqdm(total=1, desc="Training model") as pbar:
    model.fit(X_train_tfidf, y_train)
    pbar.update(1)

Training model:   0%|          | 0/1 [00:00<?, ?it/s]

In [22]:
X_test_tfidf = tfidf.transform(X_test) 

In [23]:
predictions = model.predict(X_test_tfidf)

In [24]:
f1 = f1_score(predictions, y_test)
print(f"F1-score на тренировочных данных: {f1:.4f}")

F1-score на тренировочных данных: 0.7242


## Обучение

In [25]:
from xgboost import XGBClassifier

model = XGBClassifier(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.1,
    random_state=RANDOM_STATE,
    eval_metric='logloss',  # для бинарной классификации
    use_label_encoder=False
)


In [29]:
model_2 = XGBClassifier(random_state=RANDOM_STATE)

In [30]:
model_2.fit(X_train_tfidf, y_train)

/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:49:03] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [31]:
predictions = model_2.predict(X_test_tfidf)


In [32]:
f1 = f1_score(y_test, predictions)
print(f"F1-score с XGBoost: {f1:.4f}")

F1-score с XGBoost: 0.7128


## Выводы

In [35]:
X_test_tfidf[0]

<1x142320 sparse matrix of type '<class 'numpy.float64'>'
	with 72 stored elements in Compressed Sparse Row format>

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны